## AEP raster calculation

#### This code is created to calculate AEP raster grids for Turkey and Brush Creek probabilistic modeling. The summarized workflow of how this script works is:
- It reads all the water surface elevations (WSE) raster files for all the 96 events and, the csv file containing the probability weights of each event.
- For each wse raster, it assigns the corresponding probability weight value to all the grids/cells that are wetted (i.e., cells having wse values).
- It sums up all the wse rasters containing their corresponding probability weight values.
- Finally, the summed up raster is then converted to a .tif file.

In [6]:
import os
import glob
import pathlib as pl

import numpy as np
import pandas as pd
import geopandas as gpd
from osgeo import gdal

In [7]:
def getTifData(tif_path: str):
    """Read a WSE raster and return the gdal objects"""
    src = gdal.Open(tif_path)
    rb = src.GetRasterBand(1)
    gt = src.GetGeoTransform()
    proj = src.GetProjection()
    return rb, gt, src

In [8]:
def calculateAEP(tif_dir, csv_path):
    """Reads all WSE raster files and Probability weights csv. Computes AEP 
    by assigning probability weights based on raster names while preserving NoData values."""

    weights_df = pd.read_csv(csv_path) 
    weights_df.set_index('wse_rasters', inplace=True) 

    tif_files = glob.glob(os.path.join(tif_dir, "*.tif"))
    
    aep_raster = None

    for tif_file in tif_files:
        raster_name = os.path.splitext(os.path.basename(tif_file))[0]
        print(f"Processing: {raster_name}")
        
        rb, gt, src = getTifData(tif_file)
        wse = rb.ReadAsArray()
        nodata_value = src.GetRasterBand(1).GetNoDataValue()

        #Initializing result array only once in beginning
        if aep_raster is None:
            aep_raster = np.zeros_like(wse, dtype=np.float32)
            final_mask = np.zeros_like(wse, dtype=bool)  #Tracking valid data

        weight = weights_df.loc[raster_name, 'weight']
        
        valid_mask = (wse != nodata_value) & (wse > 0)
        aep_raster[valid_mask] += weight
        final_mask |= valid_mask  #Accumulating valid data locations to assign NoData value

    aep_raster[~final_mask] = -9999  #NoData value

    return aep_raster

In [9]:
def AEPasTIF(aep_raster, reference_tif, output_path):
    """Saves the computed AEP values as a TIFF file using the properties of a 
    reference TIF file (i.e., one of the output WSE tif file)"""
    
    #Opening the reference TIFF to use the raster properties for the output raster
    rb, gt, src = getTifData(reference_tif)
    projection = src.GetProjection()
    geotransform = src.GetGeoTransform()
    
    rows, cols = aep_raster.shape

    #Creating a new TIFF file with same properties
    driver = gdal.GetDriverByName("GTiff")
    out_tif = driver.Create(str(output_path), cols, rows, 1, gdal.GDT_Float32)
    out_tif.SetGeoTransform(geotransform)
    out_tif.SetProjection(projection)

    #Writing AEP raster to output tif file
    out_tif.GetRasterBand(1).WriteArray(aep_raster)
    out_tif.GetRasterBand(1).SetNoDataValue(-9999)
    out_tif.FlushCache()
    out_tif = None  

    print(f"AEP raster saved to: {output_path}")

#### AEP raster calculation using the above functions

In [10]:
#Calculating AEP raster and saving it as output
root_dir = pl.Path(os.getcwd())

tif_directory = root_dir.parent/'Turkey_wse_rasters_Slop'             
weights_csv = root_dir.parent/'csv_files/event_weights_TurkeyCreek.csv'   

#Calculating AEP values
aep_val = calculateAEP(tif_directory, weights_csv)

#Saving calculated AEP values as a tif file
ref_tif_files = glob.glob(os.path.join(tif_directory, "*.tif"))
reference_tif = ref_tif_files[0]
output_path_AEP = root_dir.parent/'outputs/AEP_TurkeyCreek_Slop1.tif'

AEPasTIF(aep_val, reference_tif, output_path_AEP)

Processing: 1000_L_q1
Processing: 1000_L_q2
Processing: 1000_L_q3
Processing: 1000_L_q4
Processing: 1000_U_q1
Processing: 1000_U_q2
Processing: 1000_U_q3
Processing: 1000_U_q4
Processing: 100_L_q1
Processing: 100_L_q2
Processing: 100_L_q3
Processing: 100_L_q4
Processing: 100_U_q1
Processing: 100_U_q2
Processing: 100_U_q3
Processing: 100_U_q4
Processing: 10_L_q1
Processing: 10_L_q2
Processing: 10_L_q3
Processing: 10_L_q4
Processing: 10_U_q1
Processing: 10_U_q2
Processing: 10_U_q3
Processing: 10_U_q4
Processing: 2000_L_q1
Processing: 2000_L_q2
Processing: 2000_L_q3
Processing: 2000_L_q4
Processing: 2000_U_q1
Processing: 2000_U_q2
Processing: 2000_U_q3
Processing: 2000_U_q4
Processing: 200_L_q1
Processing: 200_L_q2
Processing: 200_L_q3
Processing: 200_L_q4
Processing: 200_U_q1
Processing: 200_U_q2
Processing: 200_U_q3
Processing: 200_U_q4
Processing: 25_L_q1
Processing: 25_L_q2
Processing: 25_L_q3
Processing: 25_L_q4
Processing: 25_U_q1
Processing: 25_U_q2
Processing: 25_U_q3
Processing: 

### END